In [6]:
!echo $PYTHONPATH

/global/homes/w/wlou1991/GitHub/openPMD-beamphysics/::/global/homes/w/wlou1991/CSR2D/


In [4]:
from csr2d.dist import lambda_p_Gauss

In [5]:
lambda_p_Gauss(0.0001, 0.0001)

mpf('-5.9206848026113122e-29')

In [ ]:
from csr2d.core import psi_s, Ws

In [ ]:
psi_s(0,0.002,1000000000000000)

In [ ]:
import numpy
numpy.pi.__class__

In [ ]:
import matplotlib.pyplot as plt

gamma = 500
rho = 1
sigmax = 10E-6
sigmaz = 10E-6
dz = 0.09*sigmaz
dx = 0.09*sigmax

zvec,xvec, WsConv = Ws(gamma,rho,sigmaz,sigmax,dz,dx)

In [ ]:
xcenix = int(mp.floor(xvec.shape[0]/2)) # find the index for which xvec[index] is closest to zero
plt.plot(zvec/sigmaz, WsConv[:,int(xcenix)]/1e6, '-', color='black', label=f"x= {xvec[xcenix]/sigmax:5.2f} $\sigma_x$");
plt.legend(loc='upper left')
plt.xlabel('z/$\sigma_z$')
plt.ylabel('Ws$(10^6 m^{-2})$')